# Modelo de mezcla de gaussianas para aproximar el espacio original y latente 
Se implementa un modelo probabilistico en tensorflow probability para aproximar una distribucion de probabilidad a los dato originales y los de el espacio latente usando una mezcla de Gaussianas.




In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import tensorflow_probability as tfp

2025-12-09 18:12:00.828166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-09 18:12:00.903800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-09 18:12:00.943783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-09 18:12:00.961036: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-09 18:12:01.020034: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#preprocesamiento de datos
data = pd.read_csv('Liver_GSE14520_U133A.csv')
data.drop(['samples','type'], axis=1, inplace=True)
data.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,6.801198,4.553189,6.787790,5.430893,3.250222,6.272688,3.413405,3.374910,3.654116,3.804983,...,10.735084,10.398843,12.298551,12.270505,3.855588,3.148321,3.366087,3.199008,3.160388,3.366417
1,7.585956,4.193540,3.763183,6.003593,3.309387,6.291927,3.754777,3.587603,5.137159,8.622475,...,11.528447,11.369919,12.867048,12.560433,4.016561,3.282867,3.541994,3.548680,3.460083,3.423348
2,7.803370,4.134075,3.433113,5.395057,3.476944,5.825713,3.505036,3.687333,4.515175,12.681439,...,10.892460,10.416151,12.356337,11.888482,3.839367,3.598851,3.516791,3.484089,3.282626,3.512024
3,6.920840,4.000651,3.754500,5.645297,3.387530,6.470458,3.629249,3.577534,5.192624,11.759412,...,10.686871,10.524836,12.006596,11.846195,3.867602,3.180472,3.309547,3.425501,3.166613,3.377499
4,6.556480,4.599010,4.066155,6.344537,3.372081,5.439280,3.762213,3.440714,4.961625,10.318552,...,11.014454,10.775566,12.657182,12.573076,4.091440,3.306729,3.493704,3.205771,3.378567,3.392938


In [3]:
# Normalización de los datos
data = data.values.astype(np.float32)
data = StandardScaler().fit_transform(data)

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(data).batch(51)

I0000 00:00:1765303931.301558    2915 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765303931.611344    2915 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765303931.611383    2915 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765303931.616925    2915 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765303931.616977    2915 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [5]:
input_dim = data.shape[1]

In [6]:
tfd = tfp.distributions

locs = tf.Variable(tf.random.normal([3, input_dim])) # inicializamos las medias en valores aleatorios de una distribucion normal
scales = tf.Variable(tf.ones([3, input_dim]))        # inicializamos el ancho de las distribuciones en 1
logits = tf.Variable(tf.zeros([3]))                  # inicializamos los pesos de la mezcla en 0

def MixOfGaussians():
    return tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(logits=logits),
        components_distribution=tfd.MultivariateNormalDiag( 
            loc=locs,
            scale_diag=tf.nn.softplus(scales) #aseguramos desviación estandar >0
        )
    )

optimizer = tf.optimizers.Adam(learning_rate=0.05)


@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        MoG = MixOfGaussians()
        loss = -tf.reduce_mean(MoG.log_prob(batch))
    gradients = tape.gradient(loss, [locs, scales, logits])
    optimizer.apply_gradients(zip(gradients, [locs, scales, logits]))
    return loss

In [7]:
for epoch in range(30):  
    for batch in dataset:
        loss = train_step(batch)
    print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")

'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring feature)
'+ptx85' is not a recognized feature for this target (ignoring f

Epoch 0, Loss: 34724.3594
Epoch 1, Loss: 32025.0215
Epoch 2, Loss: 30198.1758
Epoch 3, Loss: 29000.5117
Epoch 4, Loss: 28215.2207
Epoch 5, Loss: 27611.5293
Epoch 6, Loss: 27075.0586
Epoch 7, Loss: 26575.5078


2025-12-09 18:12:13.878427: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8, Loss: 26070.8535
Epoch 9, Loss: 25612.2969
Epoch 10, Loss: 25246.4453
Epoch 11, Loss: 24969.8086
Epoch 12, Loss: 24709.9121
Epoch 13, Loss: 24502.4297
Epoch 14, Loss: 24392.2617


2025-12-09 18:12:14.131961: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 15, Loss: 24340.6328
Epoch 16, Loss: 24322.7559
Epoch 17, Loss: 24319.7598
Epoch 18, Loss: 24323.6855
Epoch 19, Loss: 24326.8477
Epoch 20, Loss: 24329.8574
Epoch 21, Loss: 24326.8105
Epoch 22, Loss: 24325.5176
Epoch 23, Loss: 24322.6074
Epoch 24, Loss: 24319.3086
Epoch 25, Loss: 24318.8770
Epoch 26, Loss: 24316.9336
Epoch 27, Loss: 24316.3809
Epoch 28, Loss: 24315.7148
Epoch 29, Loss: 24315.1367
